# Housing - exploratory data analysis (VISUALIZE)

NYC housing dataset b/w in 2012-13, seperated by boroughs

use created OUTPUT file at EDA stage 

- [ ] get an intuitive understanding, 
- [ ] clear data
- [ ] adjust columns data types
- [ ] adjust columns
- [ ] enrich
- [ ] outlier and missing values
- [x] visualize
- [x] make comparisons
- [ ] then **create an OUTPUT file for other models**

### possible deletion columns: 
   - borough, building_class_category .. (extracted into diff. columns)
   - block, lot (nonsense)
   - easement (%100 empty), 
   - apartment_number (%74 empty)
   - 50% of estates has 0 land_square_feet
   - 50% of estates has 0 gross_square_feet

### domain knowledge 
  - building_class_at_present: may be mapped into 3 groups to use later ?
  - building_class_at_present vs building_class_at_sale change has any meaning?
  - tax_class_at_present vs tax_class_at_sale has any meaning? (1, 2 and 4 is dominating values)
  - 10019 zipcode is %7 of houses, and 70K houses are located in of 100 zipcodes
  - 50% of estates has 0 land_square_feet and gross_square_feet. can be groupped into less categories?
  - 4000+ diff. apartment_number in 85K rows.
  - residential_units (165 types), %50 is 0 and 1 (40K+ rows) and 30K+ NaN. can be grouped into less categories?
  - commercial_units (23 types), 0 has 20K+, NaN is 60K+
  - can residential_units and commercial_units be used meaningful dummy column?
  - total_units (171 types), 1 (25K+, 0 (16K+), 2 (8K+) and NaN is 30K
     - total = residental + commercial ?
     - if there are dif, can we set to commercial or we should delete  or we may use to create a dummy category ?
     - if total = 0 , what we should do?

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

import decimal

sns.set()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)

In [8]:
raw_data = pd.read_csv('dataset/housing/output_after_eda.csv', \
                       dtype={"building_class": object, "tax_class_at_sale": object}, \
                       parse_dates=['sale_date'], infer_datetime_format=True)



raw_data.head()

,borough_name,neighborhood,building_class,building_class_name,tax_class_at_present,tax_class_at_sale,block,lot,building_class_at_present,building_class_at_sale,address,zipcode,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,sale_date,sale_price,currency
0,Manhattan,NaN,13,CONDOS - ELEVATOR APARTMENTS,,2,738,1306,,R4,345 WEST 14TH STREET,10014,0.0,NaN,0.0,0,0,0,2013-05-20,2214693.0,USD
1,Manhattan,NaN,13,CONDOS - ELEVATOR APARTMENTS,,2,738,1307,,R4,345 WEST 14TH STREET,10014,0.0,NaN,0.0,0,0,0,2013-05-16,1654656.0,USD
2,Manhattan,NaN,13,CONDOS - ELEVATOR APARTMENTS,,2,738,1308,,R4,345 WEST 14TH STREET,10014,0.0,NaN,0.0,0,0,0,2013-05-23,1069162.0,USD
3,Manhattan,NaN,13,CONDOS - ELEVATOR APARTMENTS,,2,738,1309,,R4,345 WEST 14TH STREET,10014,0.0,NaN,0.0,0,0,0,2013-05-15,1374637.0,USD
4,Manhattan,NaN,13,CONDOS - ELEVATOR APARTMENTS,,2,738,1310,,R4,345 WEST 14TH STREET,10014,0.0,NaN,0.0,0,0,0,2013-05-13,1649565.0,USD


In [9]:
raw_data.dtypes

borough_name                         object
neighborhood                         object
building_class                       object
building_class_name                  object
tax_class_at_present                 object
tax_class_at_sale                    object
block                                 int64
lot                                   int64
building_class_at_present            object
building_class_at_sale               object
address                              object
zipcode                               int64
residential_units                   float64
commercial_units                    float64
total_units                         float64
land_square_feet                      int64
gross_square_feet                     int64
year_built                            int64
sale_date                    datetime64[ns]
sale_price                          float64
currency                             object
dtype: object

In [10]:
raw_data.describe(include='all')

,borough_name,neighborhood,building_class,building_class_name,tax_class_at_present,tax_class_at_sale,block,lot,building_class_at_present,building_class_at_sale,address,zipcode,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,sale_date,sale_price,currency
count,85975,85950,81771,81771,85975,85975,85975.000000,85975.000000,85975,85975,85975,85975.000000,56246.000000,23583.000000,56246.000000,8.597500e+04,8.597500e+04,85975.000000,85975,8.597500e+04,85975
unique,5,256,39,39,11,8,NaN,NaN,170,170,63419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,386,NaN,1
top,Manhattan,MIDTOWN WEST,01,ONE FAMILY HOMES,2,2,NaN,NaN,D4,D4,870 7 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-09-27 00:00:00,NaN,USD
freq,27395,6264,14846,14846,32899,36960,NaN,NaN,13461,13461,2087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,988,NaN,85975
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-01 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-26 00:00:00,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,3661.041721,405.420541,NaN,NaN,NaN,10758.035254,1.956850,0.150702,2.307666,3.085275e+03,5.049742e+03,1681.403303,NaN,8.850979e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,3404.028873,652.166743,NaN,NaN,NaN,662.101399,11.571244,3.237054,12.318554,5.501497e+04,2.836546e+04,670.796500,NaN,9.424213e+06,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN,0.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,1052.000000,23.000000,NaN,NaN,NaN,10028.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1910.000000,NaN,0.000000e+00,NaN


# cross analysis

In [11]:
k = raw_data[['building_class_name', 'residential_units']] 


k.groupby([ 'building_class_name', 'residential_units']).size()#.sort_values(ascending=False).unstack('residential_units')#.plot(kind='pie')

building_class_name                      residential_units
ASYLUMS AND HOMES                        0.0                      3
                                         2.0                      1
                                         40.0                     1
                                         65.0                     1
COMMERCIAL CONDOS                        0.0                   1194
COMMERCIAL GARAGES                       0.0                    506
                                         1.0                      5
                                         2.0                      1
COMMERCIAL VACANT LAND                   0.0                    253
                                         9.0                      2
                                         20.0                     2
                                         105.0                    1
CONDO-RENTALS                            0.0                      4
                                         8.0             